In [ ]:
! unzip -q "/content/drive/MyDrive/Colab Notebooks/Human_Activity_Recognition/imageDataset.zip"

In [ ]:
! pip install ultralytics==8.0.186
! pip install supervision
!pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'
from IPython import display
display.clear_output()

In [ ]:
!mkdir -p weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P weights

In [ ]:
from ultralytics import YOLO
import supervision as sv
import numpy as np
import cv2
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import torch
from glob import glob
from tqdm.auto import tqdm
import os

In [ ]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

In [ ]:
CHECKPOINT_PATH = "weights/sam_vit_h_4b8939.pth"
sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)
mask_predictor = SamPredictor(sam)

In [ ]:
# MODEL
MODEL = "yolov8x-pose.pt"
yolo_model = YOLO(MODEL)

100%|██████████| 133M/133M [00:01<00:00, 89.0MB/s]


In [ ]:
DEVICE

device(type='cuda', index=0)

In [ ]:
def get_mask(img):
    try:
        yolo_resluts = yolo_model(img)[0]
        yolo_detections = sv.Detections.from_ultralytics(yolo_resluts)
        mask_predictor.set_image(img)
        bool_masks = []
        for box in yolo_detections.xyxy:
            masks, scores, logits = mask_predictor.predict(
                box=box,
                multimask_output=True
            )
            sam_detections = sv.Detections(
                xyxy=sv.mask_to_xyxy(masks=masks),
                mask=masks
            )
            sam_detections = sam_detections[sam_detections.area == np.max(sam_detections.area)]
            bool_masks.append(sam_detections[0].mask[0])
        if len(bool_masks)>1:
            mask = bool_masks[0] | bool_masks[1]
        elif len(bool_masks)==1:
            mask = bool_masks[0]
    except Exception as e:
        print(e)
        return np.array([])
    return mask


In [ ]:
def save_binary_segmentation_img(mask, dest_path):
    binary_mask_img = mask.astype(np.int8)*255
    cv2.imwrite(dest_path,binary_mask_img)

def save_rgb_segmentation_img(img, mask, dest_path):
    image_bgr_mask_marg = cv2.bitwise_and(img, img, mask=mask.astype(np.int8))
    cv2.imwrite(dest_path,image_bgr_mask_marg)


In [ ]:
source_dir = "imageDataset/Clapping"
dist_binary_dir = "dataset_segmentation_binary/Clapping"
dist_rgb_dir = "dataset_segmentation_rgb/Clapping"

In [ ]:
img_paths = glob(f"{source_dir}/*/*.jpg")
len(img_paths)

2920

In [ ]:
max_workers = 4  # Maximum number of threads
from concurrent.futures import ThreadPoolExecutor, as_completed

def process_image(img_path, source_dir, dist_binary_dir, dist_rgb_dir):
    '''
    This function will process each image, including mask generation and saving binary/rgb segmented images.
    '''
    try:
        # Generate destination paths for binary and RGB segmented images
        dist_binary_path = img_path.replace(source_dir, dist_binary_dir)
        dist_rgb_path = img_path.replace(source_dir, dist_rgb_dir)

        # Skip if the images already exist
        if os.path.exists(dist_binary_path) and os.path.exists(dist_rgb_path):
            return

        # Create directories if they don't exist
        os.makedirs(os.path.dirname(dist_binary_path), exist_ok=True)
        os.makedirs(os.path.dirname(dist_rgb_path), exist_ok=True)

        # Read image
        img = cv2.imread(img_path)

        # Get mask for the image
        mask = get_mask(img)

        # If no mask is generated, save a black image and return
        if len(mask) == 0:
            black_img = np.zeros_like(img)
            cv2.imwrite(dist_binary_path, black_img)
            cv2.imwrite(dist_rgb_path, black_img)
            return

        # Save binary and RGB segmented images
        save_binary_segmentation_img(mask, dist_binary_path)
        save_rgb_segmentation_img(img, mask, dist_rgb_path)

    except Exception as e:
        print(f"Error processing {img_path}: {e}")

def process_all_images(img_paths, source_dir, dist_binary_dir, dist_rgb_dir):
    '''
    This function handles multithreading and processes all images with a progress bar.
    '''
    # Use ThreadPoolExecutor to run the process_image function in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit tasks for all image paths
        futures = [
            executor.submit(process_image, img_path, source_dir, dist_binary_dir, dist_rgb_dir)
            for img_path in img_paths
        ]

        # Track the progress using tqdm
        for _ in tqdm(as_completed(futures), total=len(futures), desc="Processing Images"):
            pass




In [ ]:
process_all_images(img_paths, source_dir, dist_binary_dir, dist_rgb_dir)

Processing Images:   0%|          | 0/2920 [00:00<?, ?it/s]


Ultralytics YOLOv8.0.186 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Ultralytics YOLOv8.0.186 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Ultralytics YOLOv8.0.186 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


bn
bn
bn



Ultralytics YOLOv8.0.186 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-pose summary (fused): 287 layers, 69462204 parameters, 0 gradients
YOLOv8x-pose summary (fused): 287 layers, 69462204 parameters, 0 gradients
YOLOv8x-pose summary (fused): 287 layers, 69462204 parameters, 0 gradients
YOLOv8x-pose summary (fused): 287 layers, 69462204 parameters, 0 gradients
WARNING ⚠️ NMS time limit 0.550s exceeded
WARNING ⚠️ NMS time limit 0.550s exceeded
WARNING ⚠️ NMS time limit 0.550s exceeded
WARNING ⚠️ NMS time limit 0.550s exceeded
0: 480x640 1 person, 283.5ms
Speed: 0.5ms preprocess, 70.9ms inference, 176.1ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 person, 208.8ms
Speed: 0.3ms preprocess, 52.2ms inference, 177.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 person, 218.1ms
Speed: 0.9ms preprocess, 54.5ms inference, 193.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 person, 211.3ms
Speed: 2.4ms preprocess, 52.8m

repeat_interleave() received an invalid combination of arguments - got (NoneType, int, dim=int), but expected one of:
 * (Tensor input, Tensor repeats, int dim = None, *, int output_size = None)
 * (Tensor repeats, *, int output_size = None)
      didn't match because some of the keywords were incorrect: dim
 * (Tensor input, int repeats, int dim = None, *, int output_size = None)



...


0: 480x640 1 person, 251.5ms
0: 480x640 1 person, 111.0ms
Speed: 59.9ms preprocess, 111.0ms inference, 255.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 226.2ms
Speed: 3.3ms preprocess, 226.2ms inference, 492.4ms postprocess per image at shape (1, 3, 480, 640)



repeat_interleave() received an invalid combination of arguments - got (NoneType, int, dim=int), but expected one of:
 * (Tensor input, Tensor repeats, int dim = None, *, int output_size = None)
 * (Tensor repeats, *, int output_size = None)
      didn't match because some of the keywords were incorrect: dim
 * (Tensor input, int repeats, int dim = None, *, int output_size = None)



0: 480x640 1 person, 171.5ms
Speed: 117.9ms preprocess, 171.5ms inference, 184.0ms postprocess per image at shape (1, 3, 480, 640)


0: 480x640 1 person, 167.0ms
Speed: 116.0ms preprocess, 167.0ms inference, 711.2ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 person, 207.7ms
Speed: 0.8ms preprocess, 103.8ms inference, 90.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 123.0ms
Speed: 43.3ms preprocess, 123.0ms inference, 446.1ms postprocess per image at shape (1, 3, 480, 640)



repeat_interleave() received an invalid combination of arguments - got (NoneType, int, dim=int), but expected one of:
 * (Tensor input, Tensor repeats, int dim = None, *, int output_size = None)
 * (Tensor repeats, *, int output_size = None)
      didn't match because some of the keywords were incorrect: dim
 * (Tensor input, int repeats, int dim = None, *, int output_size = None)



0: 480x640 1 person, 226.5ms
Speed: 2.8ms preprocess, 226.5ms inference, 494.1ms postprocess per image at shape (1, 3, 480, 640)


0: 480x640 1 person, 177.2ms
0: 480x640 1 person, 186.4ms
Speed: 6.1ms preprocess, 93.2ms inference, 241.4ms postprocess per image at shape (1, 3, 480, 640)
Speed: 50.7ms preprocess, 88.6ms inference, 30.1ms postprocess per image at shape (1, 3, 480, 640)


0: 480x640 1 person, 109.0ms
0: 480x640 1 person, 221.3ms
Speed: 1.8ms preprocess, 221.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)


0: 480x640 1 person, 152.9ms
Speed: 68.6ms preprocess, 152.9ms inference, 282.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 person, 221.5ms
Speed: 28.2ms preprocess, 110.7ms inference, 58.1ms postprocess per image at shape (1, 3, 480, 640)


0: 480x640 1 person, 231.2ms
Speed: 2.4ms preprocess, 231.2ms inference, 294.9ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 person, 181.4ms
Speed: 31.5ms preprocess, 90.7ms i

In [ ]:
! zip -q -r "dataset_segmentation_binary.zip" "dataset_segmentation_binary"
! zip -q -r "dataset_segmentation_rgb.zip" "dataset_segmentation_rgb"

In [ ]:
! cp "dataset_segmentation_binary.zip" "/content/drive/MyDrive/Colab Notebooks/Human_Activity_Recognition"
! cp "dataset_segmentation_rgb.zip" "/content/drive/MyDrive/Colab Notebooks/Human_Activity_Recognition"